# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [3]:
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt//pvl?uf=RJ&tipo_interessado=Estado"

In [4]:
r = requests.get(url)


In [5]:
r.status_code

200

In [6]:
data_json = r.json()

In [7]:
print(data_json['items'])

[{'id_pleito': 10002, 'tipo_interessado': 'Estado', 'interessado': 'Rio de Janeiro', 'cod_ibge': 33, 'uf': 'RJ', 'num_pvl': None, 'status': 'Encaminhado à PGFN com manifestação técnica favorável', 'num_processo': '17944.001097/2009-67', 'data_protocolo': '2009-09-09T00:00:00Z', 'tipo_operacao': 'Operação contratual externa (com garantia da União)', 'finalidade': 'Profisco', 'tipo_credor': 'Instituição Financeira Internacional', 'credor': 'Banco Interamericano de Desenvolvimento', 'moeda': 'Dólar dos EUA', 'valor': 19759050, 'pvl_assoc_divida': 1, 'pvl_contradado_credor': 0, 'data_status': '28/09/2009'}, {'id_pleito': 9750, 'tipo_interessado': 'Estado', 'interessado': 'Rio de Janeiro', 'cod_ibge': 33, 'uf': 'RJ', 'num_pvl': None, 'status': 'Deferido', 'num_processo': '17944.001046/2006-92', 'data_protocolo': '2006-07-07T00:00:00Z', 'tipo_operacao': 'Operação contratual interna', 'finalidade': 'Infraestrutura', 'tipo_credor': 'Instituição Financeira Nacional', 'credor': 'Banco Nacional d

In [8]:
df = pd.DataFrame(data_json['items'])

In [10]:
# ver as primeiras linhas e tipos
display(df.head())
df.dtypes

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,10002,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001097/2009-67,2009-09-09T00:00:00Z,Operação contratual externa (com garantia da U...,Profisco,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,19759050.0,1,0,28/09/2009
1,9750,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.001046/2006-92,2006-07-07T00:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,305737280.0,1,0,13/07/2006
2,11306,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001408/2009-98,2009-11-23T00:00:00Z,Operação contratual externa (com garantia da U...,Fortalecimento Institucional,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,485000000.0,1,0,25/11/2009
3,10673,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001256/2010-67,2011-06-03T00:00:00Z,Operação contratual interna (com garantia da U...,PAC - Copa Estádios,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,400000000.0,1,0,04/07/2011
4,7080,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000492/2012-28,2012-04-16T00:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,300000000.0,1,0,03/09/2012


id_pleito                  int64
tipo_interessado          object
interessado               object
cod_ibge                   int64
uf                        object
num_pvl                   object
status                    object
num_processo              object
data_protocolo            object
tipo_operacao             object
finalidade                object
tipo_credor               object
credor                    object
moeda                     object
valor                    float64
pvl_assoc_divida           int64
pvl_contradado_credor      int64
data_status               object
dtype: object

In [37]:
# Parte 2 — top 3 status (usando todos os registros com uf == 'RJ')
df_rj = df[df['uf'].astype(str).str.upper() == 'RJ'].copy()

df_rj['status_clean'] = (
    df_rj['status']
    .astype(str)
    .str.normalize('NFKC')
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)
df_rj.loc[df_rj['status_clean'] == '', 'status_clean'] = pd.NA

top3_status = df_rj['status_clean'].value_counts(dropna=False).head(3)
print("Top 3 status (valor : contagem):")
print(top3_status)

Top 3 status (valor : contagem):
status_clean
Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Name: count, dtype: int64


In [38]:
# Parte 3 
df_rj['data_status_str'] = df_rj['data_status'].astype(str)

df_rj['data_status_parsed'] = pd.to_datetime(df_rj['data_status_str'], dayfirst=True, errors='coerce')

# ano a partir do parsed (seguro para .dt)
year_from_parsed = df_rj['data_status_parsed'].dt.year.astype('Int64')

# fallback: extrair 4 dígitos do texto quando parsing falhar
year_from_text = df_rj['data_status_str'].str.extract(r'(\d{4})', expand=False)

# combinar: prioriza parsed; se faltar, usa texto extraído
df_rj['ano_status'] = year_from_parsed
mask_fill = df_rj['ano_status'].isna() & year_from_text.notna()
df_rj.loc[mask_fill, 'ano_status'] = year_from_text[mask_fill].astype('Int64')

# mostrar amostra para verificação
print(df_rj[['data_status', 'data_status_parsed', 'ano_status']].head(10))

  data_status data_status_parsed  ano_status
0  28/09/2009         2009-09-28        2009
1  13/07/2006         2006-07-13        2006
2  25/11/2009         2009-11-25        2009
3  04/07/2011         2011-07-04        2011
4  03/09/2012         2012-09-03        2012
5  04/07/2011         2011-07-04        2011
6  27/06/2002         2002-06-27        2002
7  04/10/2012         2012-10-04        2012
8  26/02/2016         2016-02-26        2016
9  18/04/2008         2008-04-18        2008


In [39]:
# Parte 4 — frequência por ano do campo ano_status
freq_anos = df_rj['ano_status'].value_counts(dropna=True).sort_index()
print("Frequência por ano (ano_status):")
print(freq_anos)

Frequência por ano (ano_status):
ano_status
2002     4
2006     2
2008     2
2009     9
2010     3
2011     6
2012    11
2013     7
2014     4
2016     4
Name: count, dtype: Int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [64]:
def consulta_pvl(uf: str, tipo_interessado: str, *, timeout: int = 30, page_size: int = 1000) -> pd.DataFrame:
   
    base = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    uf = str(uf).strip().upper()
    tipo_interessado = str(tipo_interessado).strip()

    all_items = []
    offset = 0

    while True:
        params = {"uf": uf, "tipo_interessado": tipo_interessado, "limit": page_size, "offset": offset}
        resp = requests.get(base, params=params, timeout=timeout)
        resp.raise_for_status()
        data = resp.json()

        items = data.get("items", [])
        if not items:
            break

        all_items.extend(items)

        if len(items) < page_size:
            break

        offset += page_size

    return pd.DataFrame(all_items)

In [65]:
# mostra os valores únicos de status e quantos aparecem
print("Total registros:", len(df_mg_estado))
print("\nValores únicos em status (até 200):")
print(df_mg_estado['status'].drop_duplicates().head(200).to_list())

print("\nContagem por status (top 50):")
print(df_mg_estado['status'].value_counts(dropna=False).head(50))

Total registros: 41

Valores únicos em status (até 200):
['Arquivado', 'Encaminhado à PGFN com manifestação técnica favorável', 'Arquivado a pedido', 'Arquivado pela STN', 'Arquivado por decurso de prazo', 'Deferido', 'Indeferido']

Contagem por status (top 50):
status
Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  8
Arquivado a pedido                                        5
Arquivado                                                 3
Arquivado pela STN                                        3
Arquivado por decurso de prazo                            1
Indeferido                                                1
Name: count, dtype: int64


In [69]:
df_mg_estado = consulta_pvl("MG", "Estado")

qtd_arquivado = (df_mg_estado['status'].astype(str).str.strip() == "Arquivado por decurso de prazo").sum()
print(qtd_arquivado)

1


In [67]:
df_ba = consulta_pvl("BA", "Estado") 

df_ba_deferido = df_ba[df_ba['status'].astype(str).str.strip() == "Deferido"].copy()

if df_ba_deferido.empty:
    print("Nenhum registro 'Deferido' encontrado para BA com o tipo/consulta usada.")
else:
    counts = df_ba_deferido.groupby('interessado', dropna=True).size()
    top_mun = counts.sort_values(ascending=False).head(1)
    print(top_mun)

interessado
Bahia    57
dtype: int64


In [68]:
# buscar e salvar
df_ba_estado = consulta_pvl("BA", "Estado")
# opcional: garantir ordem/colunas desejadas antes de salvar
df_ba_estado.to_csv("solicitacoes_ba_estado.csv", index=False, encoding="utf-8")
print("Arquivo salvo: solicitacoes_ba_estado.csv")

Arquivo salvo: solicitacoes_ba_estado.csv
